# Plotting the list of existing GP's of Pescara into a geographic map

## hetting the excel data and put into pandas frame


In this notbook I will show how to scrape table data from the following website:  
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
    

### First we have to import the following Libraries 

In [168]:
import pandas as pd # the pandas package
import numpy as np # the numpy package
import geocoder # import geocoder
#!pip install folium
import requests # retrieve function used to get the nearby venues
import folium # map rendering library
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

### reading the excel file





In [84]:
filename = 'studio-medico.xlsx'

df = pd.read_excel(filename)
df = df.drop('Unnamed: 0', 1)
df.dropna(inplace=True, )
df.reset_index(drop=True, inplace=True)

df.head(10)


,Unnamed: 1
0,Accorona Attilio
1,"Via Regina Margherita, 125, 65124 Pescara"
2,Distretto: Area Distrettuale Pescara
3,Tel.: 085 4227468
4,E-mail: attilioaccorona@libero.it
5,lunedì 9.15-11 16-17
6,martedì 9.15-11
7,mercoledì 9-11
8,giovedì 9-11
9,venerdì 9.15-11 16-17


In [127]:
# show the dataframe

# substring to be searched
sub ='E-mail:'
 
lijst = df.loc[df['Unnamed: 1'].str.contains('E-mail:')]  
medici=pd.DataFrame()
for waarde, row in lijst.iterrows():
    #print(waarde, row['Unnamed: 1'])
    
    #build new dataframe
    #print(waarde)
    naam = df.iloc[waarde-4]['Unnamed: 1']
    address =  df.iloc[waarde-3]['Unnamed: 1']
    district = df.iloc[waarde-2]['Unnamed: 1']
    phone = df.iloc[waarde-1]['Unnamed: 1']
    email = df.iloc[waarde]['Unnamed: 1']

    #print(naam,'--',address,'--',district,'--',phone,'--',email)
    dict = {'Name': naam, 'Address': address, 'District': district, 'Phone': phone, 'E-mail': email}
    medici = medici.append(dict, ignore_index = True)
    
medici.head(5)


,Address,District,E-mail,Name,Phone
0,"Via Regina Margherita, 125, 65124 Pescara",Distretto: Area Distrettuale Pescara,E-mail: attilioaccorona@libero.it,Accorona Attilio,Tel.: 085 4227468
1,"Via Malagrida, 9, 65124 Pescara",Distretto: Area Distrettuale Pescara,E-mail: attilioaccorona@libero.it,Accorona Attilio,Tel.: 085 8967153
2,"Via Ravenna, 11, 65012 Cepagatti",Distretto: Area Metropolitana,E-mail: auralambro@gmail.com,AMBROSINI LAURA,Tel.: 085 9774149
3,"Via Palermo, 8, 65122 Pescara",Distretto: Area Distrettuale Pescara,E-mail: amorosodr.patriz@gmail.com,Amoroso Patrizia,Tel.: 085 4216700
4,"Via Adda, 16 65015, 65015 Montesilvano",Distretto: Area Distrettuale Metropolitana,E-mail: andreassimarinellif@gmail.com,Andreassi Marinelli Francesco,Tel.: 085 4454552


In [158]:
# add Geographical data to data frame

g = geocoder.arcgis("453 Booth Street, Ottawa ON")

waarde = medici['Address'].iloc[0]
print(waarde)
g = geocoder.arcgis(waarde)
print(g.latlng)
# create a function to retreive the latitude and longitude based on postal code

def get_latlon(address):

    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis(address)
      lat_lng_coords = g.latlng
    print(lat_lng_coords)
        
    return lat_lng_coords


# initializeing dataframes
medici['Latitude'] = np.nan
medici['Longitude'] = np.nan
#for loop through the dataframe using every postalcode as imput for the function 'get_latlon'
for index in range(0,len(medici)): 
    # get postal code from index
    address = medici.at[index, 'Address']
    #retreive the lat lon by running the function 'get_latlon'
    value = get_latlon(address)
    #addig the result to the columns in the dataframe
    medici.at[index, 'Latitude'] = value[0]
    medici.at[index, 'Longitude'] = value[1]
print('Postal Code conversion to geographical coordinates completed.')

Via Regina Margherita, 125, 65124 Pescara
[42.478230019322694, 14.201510015037627]
[42.478230019322694, 14.201510015037627]
[42.46302000827296, 14.200799984019625]
[42.383209998897854, 14.12362000631336]
[42.46632997992742, 14.210830020816445]
[42.50274000340269, 14.170810037372092]
[42.455459992613385, 14.217779959653399]
[42.465120010294555, 14.102199968929483]
[42.26710999490092, 13.994949997400369]
[42.30599613105225, 13.931114751210773]
[42.5063800124947, 14.157479961476042]
[42.44726999311115, 14.217170008559322]
[42.49714998644643, 14.129009989150347]
[42.49714998644643, 14.129009989150347]
[42.51319331975251, 14.155710257214237]
[42.43751000000003, 14.137350000000026]
[42.42486998551752, 14.158819976336702]
[42.26710999490092, 13.994949997400369]
[42.26978000824795, 14.00751002785168]
[42.294922871937025, 13.968759252591784]
[42.469630019026624, 14.213039993407676]
[42.21384999911015, 13.82692997056381]
[42.26978998271272, 14.00751002785168]
[42.26710999490092, 13.9949499974003

[42.47380005776594, 14.19912008298519]
[42.46952000654747, 14.178739988328289]
[42.45753112175715, 13.922113141738054]
[42.4743199872199, 13.991760012689866]
[42.51011000131729, 14.153299990181495]
[42.36574999166919, 14.07405001766702]
[42.38307999557958, 14.124939988425439]
[42.47536998823068, 14.199510009120928]
[42.333955964468714, 13.860509357923428]
[42.34687666338903, 13.952749935358195]
[42.341329987319796, 13.896660028908713]
[42.475239984912434, 14.203280021531185]
[42.45815033341415, 14.014098341004367]
[42.449856911289096, 14.03518678477516]
[42.43506139822628, 13.98460365404512]
[42.499809489640185, 14.131892157341346]
[42.509310032478226, 14.1647701217629]
[42.499809489640185, 14.131892157341346]
[42.457902870723025, 13.924147033552906]
[42.45876182065425, 13.92662739343746]
[42.50648998306434, 14.139240018156926]
[42.45540999456094, 14.217779959653399]
[42.462879988580426, 14.215880033660625]
[42.46087998266373, 14.21677999860458]
[42.46400999485701, 14.09912003860768]
[

In [178]:
#reformatting
medici = medici.groupby(['Address','E-mail','District'])['Name'].apply(', '.join).reset_index()



medici.head(15)


,Address,E-mail,District,Name
0,"C.da Cepraneto, 65010 Collecorvino",E-mail: damvolp@tin.it,Distretto: Vestina,Volpone Damiano
1,"C.da Congiunti, 65010 Collecorvino",E-mail: damvolp@tin.it,Distretto: Vestina,Volpone Damiano
2,"C.da Congiunti, 73 Via D'Annunzio, 65010 Colle...",E-mail: vag.pardi@libero.it,Distretto: Area Distrettuale Montana,Pardi Vincenzo
3,"C.so 1ø maggio, 47, 65027 Scafa",E-mail: p.giamba53@gmail.com,Distretto: Area Distrettuale Montana,Di Giambattista Paolo
4,"C.so Gramsci, 122 , 65026 Popoli",E-mail: angeloterzini@tiscali.it,Distretto: Maiella Morrone,Terzini Angelo
5,"C.so Santarelli, 83, 65024 Manoppello",E-mail: orlando_elena@virgilio.it,Distretto: Area Distrettuale Montana,Orlando Elena
6,"C.so Umberto Iø, 690, 65015 Montesilvano",E-mail: difeliceguido@alice.it,Distretto: Area Distrettuale Metropolitana,Di Felice Guido
7,"C.so Umberto, 219 , 65015 Montesilvano",E-mail: anrieve@alice.it,Distretto: Montesilvano,Rieve Anna Katharina
8,"C.so Umberto, 690, 65015 Montesilvano",E-mail: lidarezai@hotmail.it,Distretto: Montesilvano,Sedighnavaz Rezai Lida
9,"C.so Umberto,9 65019, 65012 Pianella",E-mail: evaldocrisante@virgilio.it,Distretto: Area Distrettuale Metropolitana,Crisante Evaldo


In [170]:
# safe the data frame to an Excel file for later use.
medici.to_excel("./Data/medici.xlsx",index=False) 

In [174]:
#plot on map
# create map of Toronto using latitude and longitude values
latitude = 42.47823
longitude = 14.20080
map_medici = folium.Map(location=[latitude, longitude], zoom_start=12)
counter = 0

# add markers to map
for lat, lng, name, email in zip(medici['Latitude'], medici['Longitude'], medici['Name'], medici['E-mail']):
    label = '{}, {}'.format(name, email)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_medici) 
    counter = counter + 1
    
map_medici

### Update the name of the Neighborhood to name of borough if having the value "Not assigned"

In [175]:
print(counter)

348


In [7]:
# in this loop all the neighborhoods containing value 'Not assigned' will be updated by the value of the Borough
i=[]
for i in df.loc[df['Neighborhood']=='Not assigned'].index:
    df.iloc[i]['Neighborhood'] = df.iloc[i]['Borough']

### Combining rows if multiple postal code entries exist in the table

First we perform a simple test to see if there are any rows we have to combine.

In [8]:
# test if there are postal codes listed multiple times and have to be to combined into one row
tel = df['Postal Code'].value_counts(sort=True)
tel.head()

M3K    1
M5V    1
M8Z    1
M4G    1
M3L    1
Name: Postal Code, dtype: int64

Conclusion: There are no duplicate postal code entries in the table to combine
In case (for future use) we need a way to combine them, the below code can be used.

In [9]:
# This code creates a new dataframe containing all the postal codes and groups multiple entries of the same postal codes 
# with the neighborhoods separated by a ','

df_combined = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(','.join).reset_index()
df_combined.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Saving the data

In [10]:
# safe the data frame to an Excel file for later use.
df.to_excel("./Data/dataframe.xlsx",index=False)  

### Checking for the format of the table using the .shape function and print the first twelve rows of the dataframe

In [11]:
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [12]:
# display the shape
df.shape

(103, 3)